In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
NUM_ROUNDS = 10
NUM_CLIENTS = 10
BATCH_SIZE = 10
EPOCHS_PER_ROUND = 5
LEARNING_RATE = 0.1
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-7

In [3]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert into Categorical (One-Hot Encoding)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [4]:
global_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28, 28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

Metal device set to: Apple M1


In [5]:
global_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
x_train_clients = np.array_split(x_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(x_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

In [7]:
client_models = []
for i in range(NUM_CLIENTS):
    local_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    local_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    print(acc)
    client_models.append(local_model)

2023-10-05 12:30:54.301888: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - 2s 51ms/step - loss: 0.2447 - accuracy: 0.9220
[0.24470634758472443, 0.921999990940094]
32/32 [==============================] - 2s 46ms/step - loss: 0.3730 - accuracy: 0.8930
[0.3729957938194275, 0.8930000066757202]
32/32 [==============================] - 2s 46ms/step - loss: 0.2481 - accuracy: 0.9300
[0.24812829494476318, 0.9300000071525574]
32/32 [==============================] - 2s 47ms/step - loss: 0.2814 - accuracy: 0.9210
[0.2813540995121002, 0.9210000038146973]
32/32 [==============================] - 2s 47ms/step - loss: 0.2948 - accuracy: 0.9190
[0.2947859466075897, 0.9190000295639038]
32/32 [==============================] - 2s 47ms/step - loss: 0.1900 - accuracy: 0.9520
[0.19001621007919312, 0.9520000219345093]
32/32 [==============================] - 2s 48ms/step - loss: 0.2347 - accuracy: 0.9460
[0.23470455408096313, 0.9459999799728394]
32/32 [==============================] - 2s 48ms/step - loss: 0.1636 - accuracy: 0.9530
[0.163

In [8]:
acc = global_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 14s 43ms/step - loss: 2.3631 - accuracy: 0.0713
[2.363069534301758, 0.07129999995231628]


In [9]:
# SERVER ADAM OPTIMIZER
fedadam = tf.optimizers.Adam(learning_rate=LEARNING_RATE,beta_1=BETA_1,beta_2 = BETA_2,epsilon=EPSILON)

In [10]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(NUM_CLIENTS):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [11]:
acc = global_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 13s 42ms/step - loss: 1.9583 - accuracy: 0.3536
[1.9583327770233154, 0.35359999537467957]


In [12]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer=fedadam, loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(x_train,y_train,epochs=5,batch_size=20)

Epoch 1/5
3000/3000 [==============================] - 139s 45ms/step - loss: 1.6294 - accuracy: 0.4374
Epoch 2/5
3000/3000 [==============================] - 135s 45ms/step - loss: 1.8431 - accuracy: 0.3408
Epoch 3/5
3000/3000 [==============================] - 135s 45ms/step - loss: 1.8023 - accuracy: 0.3510
Epoch 4/5
3000/3000 [==============================] - 137s 46ms/step - loss: 1.7593 - accuracy: 0.3652
Epoch 5/5
3000/3000 [==============================] - 136s 45ms/step - loss: 1.6617 - accuracy: 0.4086


In [13]:
acc = fed_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 14s 43ms/step - loss: 1.8795 - accuracy: 0.3849
[1.8795104026794434, 0.3849000036716461]
